In [69]:
import cv2
import numpy as np
import math

In [70]:
def show_image(image):
    cv2.imshow("Image",image)
    while True:
        key = cv2.waitKey(1)
        if key == 27:  
            break
    cv2.destroyAllWindows()

In [71]:
def treat_img(img):
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    blurred = cv2.GaussianBlur(gray,
                                    (15, 15), 4)

    ret, thresh = cv2.threshold(blurred,
                                        180, 255,
                                        cv2.THRESH_BINARY)

    contours, hier = cv2.findContours(thresh.copy(),
                                            cv2.RETR_TREE,
                                            cv2.CHAIN_APPROX_SIMPLE)
    #show_image(img)
    #show_image(thresh)

    return contours

In [72]:
def average_centroid(centroids):
    x = []
    y = []
    totalx = 0
    totaly = 0
    for center in centroids:
        x.append(center[0])
        y.append(center[1])
    for i in range(len(x)):
        totalx += x[i]
        totaly += y[i]
    centroid = [totalx/len(x),totaly/len(y)]
    return centroid


In [73]:
def draw_rectangles(img,i):
    H,W,colors = img.shape #get width and height
    centroids = []

    for c in contours:
        # if the contour is not sufficiently large, ignore it
        if  cv2.contourArea(c) < (H*W)/2000: #lower limit
            continue

        if  cv2.contourArea(c) > (H*W)/100: #upper limit
            continue

        # get the min area rect
        rect = cv2.minAreaRect(c)
        (x, y), (w, h), angle = rect
        aspect_ratio = max(w, h) / min(w, h)
        center = [x + 0.5*w,y + 0.5*h]

        # Assume zebra line must be long and narrow (long part must be at lease 1.5 times the narrow part).
        if (aspect_ratio > 6):
            box = cv2.boxPoints(rect)
            # convert all coordinates floating point values to int
            box = np.int0(box)
            # draw a red 'nghien' rectangle
            cv2.drawContours(img, [box], 0, (0, 0, 255), 2)
            centroids.append(center) 

    centroid = average_centroid(centroids)        
    cv2.imwrite('resultados/resultado_{}.jpg'.format(i), img)
    #cv2.imshow("rectangles", img)
    return centroid

In [74]:
def dist(tuple1, tuple2):
    deltax = tuple1[0]-tuple2[0]
    deltay = tuple1[1]-tuple2[1]
    dist = math.sqrt(deltax*deltax + deltay*deltay)
    return dist

In [75]:
def draw_rectangles_centroid(img,i,centroid):
    H,W,colors = img.shape #get width and height

    for c in contours:
        # if the contour is not sufficiently large, ignore it
        if  cv2.contourArea(c) < (H*W)/2000: #lower limit
            continue

        if  cv2.contourArea(c) > (H*W)/100: #upper limit
            continue

        # get the min area rect
        rect = cv2.minAreaRect(c)
        (x, y), (w, h), angle = rect
        aspect_ratio = max(w, h) / min(w, h)
        center = [x + 0.5*w,y + 0.5*h]


        # Assume zebra line must be long and narrow (long part must be at lease 1.5 times the narrow part).
        if (aspect_ratio > 6):
            if dist(center,centroid) < (max(H,W)/10):
                box = cv2.boxPoints(rect)
                # convert all coordinates floating point values to int
                box = np.int0(box)
                # draw a red 'nghien' rectangle
                cv2.drawContours(img, [box], 0, (0, 0, 255), 2)

     
    cv2.imwrite('resultados/resultado_{}.jpg'.format(i), img)
    #cv2.imshow("rectangles", img)

In [76]:
vidcap = cv2.VideoCapture("videos\MVI_2313.MOV")
success,img = vidcap.read()
contours = treat_img(img)
i=20
centroid = draw_rectangles(img,i)
print(centroid)

[978.2722703933716, 821.7899458169937]


In [77]:
vidcap = cv2.VideoCapture("videos\MVI_2313.MOV")
success,img = vidcap.read()
contours = treat_img(img)
i=21
draw_rectangles_centroid(img,i,centroid)